In [30]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

from glob import glob
from os.path import join

from tqdm import tqdm

from langchain.document_loaders import PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from langchain.llms import OpenAI

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

from langchain.memory import ConversationBufferMemory



In [7]:
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

# Creating ChromaDB

In [8]:
files_dir = '/Users/patrick/Documents/Pessoal/paper-rag/inputs/full'

# Use a função glob para listar todos os arquivos PDF no diretório
pdf_files = glob(f'{files_dir}/*.pdf')

docs=[]

for arquivo in tqdm(pdf_files):
    print(arquivo)

    loader = PyPDFLoader(join(files_dir, arquivo))
    docs.extend(loader.load())

  0%|          | 0/34 [00:00<?, ?it/s]

/Users/patrick/Documents/Pessoal/paper-rag/inputs/full/A comprehensive survey on machine learning for networking - Evolution, applications and research opportunities.pdf


  3%|▎         | 1/34 [00:04<02:25,  4.40s/it]

/Users/patrick/Documents/Pessoal/paper-rag/inputs/full/Backhauling 5G Small Cells - A Radio Resource Management Perspective.pdf


  6%|▌         | 2/34 [00:04<01:05,  2.06s/it]

...

 94%|█████████▍| 32/34 [01:05<00:04,  2.48s/it]

/Users/patrick/Documents/Pessoal/paper-rag/inputs/full/Exploring Network-Wide Flow Data With Flowyager.pdf


 97%|█████████▋| 33/34 [01:34<00:10, 10.32s/it]

/Users/patrick/Documents/Pessoal/paper-rag/inputs/full/Resource Allocation in NFV- A Comprehensive Survey.pdf


100%|██████████| 34/34 [01:38<00:00,  2.89s/it]


In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)

In [10]:
#!rm -Rf /Users/patrick/Documents/Pessoal/paper-rag/chroma/

embedding = OpenAIEmbeddings()
persist_directory = '/Users/patrick/Documents/Pessoal/paper-rag/chroma_full/'

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

# Model

In [13]:
llm = OpenAI(temperature=0)  # Model used

# Template

In [22]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you can't make a answer with context, just say that you don't know, don't try to make up an answer. Give the document name from where the information is extracted
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Compression retrieval method configuration

In [23]:
# Wrap our vectorstore
compressor = LLMChainExtractor.from_llm(llm)  # uses an LLMChain to extract from each document only the statements that are relevant to the query.

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")  # Retriever method using MMR retrieval to ensure variability in the answer
)

# Memory

Ref: https://github.com/langchain-ai/langchain/issues/2256

In [44]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='result'
)

# Chain assembling

In [45]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    memory=memory
)

In [90]:
question = "What is 5G?"
result = qa_chain({"query": question})

print(result["result"])
print('\n')

for source in result['source_documents']:
    print ('Source: ', source.metadata['source'].split('/')[-1])
    print('Page: ', source.metadata['page'])
    print('\n')

 5G is the upcoming fifth generation cellular technology which is expected to be rolled out by 2020. It is envisioned as the technology that will connect a huge number of end-devices and offer native support to more diverse service types with quality of service (QoS) provisioning.


Source:  Backhauling 5G Small Cells - A Radio Resource Management Perspective.pdf
Page:  0


Source:  Deep learning in mobile and wireless networking - A survey.pdf
Page:  53


Source:  Hierarchical, virtualised and distributed intelligence 5G architecture for low-latency and secure applications.pdf
Page:  3


Source:  Application of Machine Learning in Wireless Networks - Key Techniques and Open Issues.pdf
Page:  3




In [91]:
question = "What are the challenges in 5G implementation?"
result = qa_chain({"query": question})

print(result["result"])
print('\n')

for source in result['source_documents']:
    print('Source: ', source.metadata['source'].split('/')[-1])
    print('Page: ', source.metadata['page'])
    print('\n')

 The challenges in 5G implementation include resource management, networking, mobility management, localization, security, open access, virtualized security, multi-tenancy, and low latency.


Source:  Application of Machine Learning in Wireless Networks - Key Techniques and Open Issues.pdf
Page:  0


Source:  Big data-driven optimization for mobile networks toward 5G.pdf
Page:  6


Source:  Hierarchical, virtualised and distributed intelligence 5G architecture for low-latency and secure applications.pdf
Page:  1




In [92]:
question = "Can you talk more about the first challenge in 5G implementation mentioned?"
result = qa_chain({"query": question})

print(result["result"])
print('\n')

for source in result['source_documents']:
    print('Source: ', source.metadata['source'].split('/')[-1])
    print('Page: ', source.metadata['page'])
    print('\n')

 The first challenge mentioned in the context is the need for algorithmic design enhancement for resource management, networking, mobility management and localization. This is necessary to ensure that the stringent requirements of 5G are met.


Source:  Backhauling 5G Small Cells - A Radio Resource Management Perspective.pdf
Page:  0


Source:  Application of Machine Learning in Wireless Networks - Key Techniques and Open Issues.pdf
Page:  0


Source:  Big data-driven optimization for mobile networks toward 5G.pdf
Page:  6


Source:  Hierarchical, virtualised and distributed intelligence 5G architecture for low-latency and secure applications.pdf
Page:  1




In [93]:
abstract="""Modeling and simulation of a cellular network typically assumes that the target area is divided into regular hexagonal cells and mobile stations (MSs) are uniformly scattered in each cell. This implies a statistically uniform distribution of traffic load over space, but in reality the spatial traffic distribution is highly non-uniform across different cells, which calls for actual spatial traffic models. In this article, we first present the analysis of traffic measurements collected from commercial cellular networks in China, and demonstrate that the spatial distribution of the traffic density (the traffic load per unit area) can be approximated by the log-normal or Weibull distribution depending on time and space. Then we propose a spatial traffic model which generates large-scale spatial traffic variations by a sum of sinusoids that captures the characteristics of log-normally distributed and spatially correlated cellular traffic. The proposed model can be directly used to generate realistic spatial traffic patterns for cellular network simulations, such as performance evaluations of network planning and load balancing."""

question = "Find a abstract similar to the mentioned below: " + abstract
result = qa_chain({"query": question})

print(result["result"])
print('\n')

for source in result['source_documents']:
    print('Source: ', source.metadata['source'].split('/')[-1])
    print('Page: ', source.metadata['page'])
    print('Content: ', source.page_content)
    print('\n')

 This article presents an analysis of traffic measurements collected from commercial cellular networks in China, demonstrating that the spatial distribution of the traffic density can be approximated by the log-normal or Weibull distribution. The authors then propose a spatial traffic model which generates large-scale spatial traffic variations by a sum of sinusoids that captures the characteristics of log-normally distributed and spatially correlated cellular traffic. The proposed model can be used to generate realistic spatial traffic patterns for cellular network simulations, such as performance evaluations of network planning and load balancing.


Source:  Spatial modeling of the traffic density in cellular networks.pdf
Page:  0
Content:  "Modeling and simulation of a cellular network typically assumes that the target area is divided into regular hexagonal cells and mobile stations (MSs) are uniformly scattered in each cell. This implies a statistically uniform distribution of traf